# TorchSig: Getting Started
This notebook gives an overview of the terms and capabilities of TorchSig.

---

## Signals
A signal has many definitions in the digital signal processing, radio frequency, and machine learning world. Below we define precisely what we define as certain signals.

* **SignalMetadata**: Object that contains all the metadata for a Signal. There are 7 **core** metadata fields:
    * center_freq: Normalized center frequency of signal.
    * bandwidth: Normalized bandwidth of signal.
    * start_in_samples: Start time of signal in terms of IQ data array idx.
    * duration_in_samples: Duration of signal in terms of IQ data array idx.
    * snr_db: Signal-to-Noise ratio in dB.
    * class_name: Signal class name/modulation type.
    * class_index: Class index of signal in dataset, which contains a list of all modulations being used.
    
    Then there are **derived** metadata fields, which are calculate from the 7 core fields. Derived fields for example include lower_freq, upper_freq, start, stop, ect.
* **Signal**: Object that represents a single signal burst. Contains IQ data and SignalMetadata. The **SignalBuilders** are responsible for creating a Signal object.
* **DatasetSignal**: Contains the base noise floor with Signal(s) placed inside it. Also called a **sample**. This represents collecting signals over a set bandwidth.
* **DatasetDict**: Same as DatasetSignal, but metadata is a list of dictionaries (that contain SignalMetadata). This makes performing target transforms much easier. Can also be called a **sample**.

---

## Transforms

### Functionals
All core compute processes and logic parameters for transforms
* functions perform fundamental operations, but do not define parameters or random distributions/sampling of how the operations are applied or randomized
* Similiar to [Torchvision's transforms framework](https://pytorch.org/vision/0.9/transforms.html#functional-transforms)

### Signal Transforms
Applied to Signal objects, which are created from the SignalBuilders. In other words, transforms that are applied to isolated bursts before being placed on noise floor. These can be understood as transmitter effects on the Signal(s).

Note that impairments are carefully applied within the SignalBuilder in a strict order to represent actual signal processing effects.

### Dataset Transforms
Applied to DatasetSignal objects, after all the IQ data of all Signal object(s) are placed onto the noise floor. These transforms are applied to the whole IQ data, including the bursts and noise floor. In particular, these transforms correspond to wideband effects, such as channel or receiver effects.

### Dataset Impairments
Impairments are a collections of SignalTransforms and DatasetTransforms that emulate different environments.
* Level 1: Perfect environment, like inside a computer simulation. Has no transforms.
* Level 2: Cabled environment. Contains some transforms that slightly impair the signals.
* Level 3: Wireless environment. Contains many transforms that impair the signals greatly.

---

## Datasets

### Two Types: IQ vs. Spectrogram Image

#### Synthetic IQ Datasets: Narrowband and Wideband
* Narrowband
* Wideband

#### Synthetic Spectrogram Dataset: Image Datasets
* Image datasets, cropping images to create a new dataset.
* CFGSignalProtocolDataset

### Dataset Objects
These are the types of object you can instantiate in order to generate, create, or read a dataset.
* DatasetMetadata:
    * NarrowbandMetadata
    * WidebandMetadtata
* NewDataset: Dataset that can infinitely (or finite) generate signals in memory, but you cannot access previously generated samples.
    * NewNarrowband, NewWideband
* DatasetCreator: Takes in a NewDataset, and writes the dataset to disk.
    * When a dataset is written or read from disk, it can either be **raw** or **processed**.
    * **raw**: Raw IQ and signal metadata were written to disk. In other words, no transforms or target transforms were applied to the data. This means when the dataset is read back in, users can define their own transforms or target transforms to apply then. Note that impairment level cannot be changed once written to disk.
    * **processed**: IQ data and/or metadata has transforms/target transforms applied. Thsi means that when the dataset is read back into memory, users cannot apply more transforms or target transforms.
* StaticDataset: Reads in a dataset from disk. This assumes you have created a dataset using a DatasetCreator, and want to read it back in.
    * StaticNarrowband, StaticWideband


| I want... | Use |
| --------- | ----- |
| an infinite dataset | NewDataset |
| to write a dataset to disk | NewDataset, DatasetCreator |
| to load a dataset from disk | StaticDataset |
| to write a dataset to disk, so someone else can use it with their own transforms/target transforms | NewDataset (no transforms or target transforms), DatasetCreator |
| a finite dataset, where I can call previously generated samples | NewDataset, DatasetCreator, StaticDataset |
---

## Workflow

### NewDataset
The below diagram shows how a sample is generated from start to finish in a NewDataset's `__getitem__` method.

![NEWDATASET](diagrams/new_dataset_workflow.png)

### StaticDataset
The below diagram shows how a sample is read in and optionally transformed in a StaticDataset's `__getitem__` method.

![STATICDATSET](diagrams/static_dataset_workflow.png)

---